In [121]:
# Import the necessary packages
import pandas as pd
from IPython.display import display, Markdown, Latex

# Manually change inputs
datapath = "./data/test.xlsx"

input_label_list = ["axe", "sous_axe", "indic", "label4", "label5", "label6"]

In [128]:
print("Processing file...")
data = pd.read_excel(datapath)
print("\nProcessing OK!")

Processing file...

Processing OK!


In [130]:
found_labels = list(data)
print("Found labels are the£ following:")
for str in found_labels:
    print("    " + str)

Found labels are the£ following:
    axe
    sous_axe
    indic
    code_hospidiag
    libelle


In [138]:
intersection = set(found_labels).intersection(input_label_list)
intersection = list(intersection)

new_labels = []
for str in found_labels:
    if str not in intersection:
        new_labels.append(str)
        
md_str = "# Would you like to keep the following new labels ? You can also edit them before saving them. Just run the next input"
new_label_str = "[\"" + "\", \"".join(new_labels) + "\"]"

input_str = "label_list = input_label_list + " + new_labels_str

display(Markdown(md_str))
get_ipython().set_next_input(input_str)

# Would you like to keep the following new labels ? You can also edit them before saving them. Just run the next input

In [139]:
label_list = input_label_list + ["code_hospidiag", "libelle"]

In [140]:
label_list

['axe',
 'sous_axe',
 'indic',
 'label4',
 'label5',
 'label6',
 'code_hospidiag',
 'libelle']